# GLLVM Simulations

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from gllvm import GLLVM, PoissonGLM, BinomialGLM, GaussianGLM

seed = 123
device = "cuda"

torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ----------------------------------------------------------
# 1. Generate synthetic data from the ground-truth model
# ----------------------------------------------------------

gllvm0 = GLLVM(latent_dim=1, output_dim=5)
gllvm0.add_glm(PoissonGLM, idx=[0, 1], name="Poisson1")
gllvm0.add_glm(BinomialGLM, idx=[2], name="Binomial1")
gllvm0.add_glm(GaussianGLM, idx=[3, 4], name="Gaussian1")

num_samples = 20000

with torch.no_grad():
    gllvm0.wz.mul_(1.0)
    z0 = gllvm0.sample_z(num_samples)
    y0 = gllvm0.sample(z=z0)

# ----------------------------------------------------------
# 2. Build a fresh model that will be trained with VI
# ----------------------------------------------------------

gllvm = GLLVM(latent_dim=1, output_dim=5)
gllvm.add_glm(PoissonGLM, idx=[0, 1], name="Poisson1")
gllvm.add_glm(BinomialGLM, idx=[2], name="Binomial1")
gllvm.add_glm(GaussianGLM, idx=[3, 4], name="Gaussian1")

# ----------------------------------------------------------
# 3. Build the encoder q(z|y)
# ----------------------------------------------------------

class Encoder(nn.Module):
    def __init__(self, input_dim=5, latent_dim=1, hidden=32):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
        )
        self.mean = nn.Linear(hidden, latent_dim)
        self.logvar = nn.Linear(hidden, latent_dim)

    def forward(self, y):
        h = self.net(y)
        mu = self.mean(h)
        logvar = self.logvar(h)
        return mu, logvar

encoder = Encoder(input_dim=5, latent_dim=1)


# --- separate gllvm parameters ---
gllvm_scale = [gllvm.log_scale]   # this is learned by ELBO
gllvm_no_scale = []

for name, p in gllvm.named_parameters():
    if name != "log_scale":
        gllvm_no_scale.append(p)
        
        
optimizer_gllvm = optim.Adam(gllvm_no_scale, lr=1e-3)
optimizer_encoder = optim.Adam(list(encoder.parameters()) + gllvm_scale, lr=1e-3)

# ----------------------------------------------------------
# 4. VI training loop: ZQE + ELBO Encoder
# ----------------------------------------------------------

gllvm.to(device)
gllvm0.to(device)
encoder.to(device)
y0 = y0.to(device)


batch_size = 10000
num_epochs = 200

dataset = y0
n = len(dataset)

for epoch in range(num_epochs):
    perm = torch.randperm(n)
    total_elbo = 0.0

    for i in range(0, n, batch_size):
        idx = perm[i:i+batch_size].to(device)
        y = dataset[idx]

        # ======================================================
        # 1. ENCODER UPDATE (phi) using ELBO ONLY
        # ======================================================
        optimizer_encoder.zero_grad()

        # forward encoder
        mu, logvar = encoder(y)
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        z = mu + eps * std

        # decoder log-likelihood for ELBO (no detach)
        logpy_z = gllvm.log_prob(y, z=z).sum(dim=-1)

        # KL(q||p)
        kl = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim=1)

        elbo = (logpy_z - kl).mean()
        loss_elbo = -elbo

        loss_elbo.backward()
        optimizer_encoder.step()

        # ======================================================
        # 2. DECODER UPDATE (theta) using CENTERED ZQ LOSS ONLY
        # ======================================================


        # ---- sample z ~ q(z|y) but encoder is frozen ----
        with torch.no_grad():
            mu, logvar = encoder(y)
            std = torch.exp(0.5 * logvar)
            eps = torch.randn_like(std)
            z = mu + eps * std  # detached latent samples from q

        # ---- sample model-generated pairs (zq,yq) ----
        with torch.no_grad():
            zq0 = gllvm.sample_z(num_samples=len(y))   # prior
            yq = gllvm.sample(z=zq0)                  # model sample

            muq, logvarq = encoder(yq)
            stdq = torch.exp(0.5 * logvarq)
            epsq = torch.randn_like(stdq)
            zq = muq + epsq * stdq                    # detached

        # ---- compute the centered ZQ estimating function ----
        logpy_zqe  = gllvm.zq_log(y,  z=z).sum(dim=-1)
        logpy_zqe2 = gllvm.zq_log(yq, z=zq).sum(dim=-1)

        # ZQ loss = -(m_q - m_q_model)
        loss_zqe = -(logpy_zqe.mean() - logpy_zqe2.mean())

        optimizer_gllvm.zero_grad()
        loss_zqe.backward()
        optimizer_gllvm.step()

        total_elbo += elbo.item()

    print(f"Epoch {epoch+1}: ELBO={total_elbo:.2f}")




Epoch 1: ELBO=-22.61
Epoch 2: ELBO=-21.70
Epoch 3: ELBO=-21.24
Epoch 4: ELBO=-20.57
Epoch 5: ELBO=-20.18
Epoch 6: ELBO=-19.76
Epoch 7: ELBO=-19.30
Epoch 8: ELBO=-18.84
Epoch 9: ELBO=-18.49
Epoch 10: ELBO=-18.06
Epoch 11: ELBO=-17.75
Epoch 12: ELBO=-17.48
Epoch 13: ELBO=-17.18
Epoch 14: ELBO=-16.94
Epoch 15: ELBO=-16.57
Epoch 16: ELBO=-16.27
Epoch 17: ELBO=-16.07
Epoch 18: ELBO=-15.81
Epoch 19: ELBO=-15.66
Epoch 20: ELBO=-15.45
Epoch 21: ELBO=-15.31
Epoch 22: ELBO=-15.18
Epoch 23: ELBO=-15.04
Epoch 24: ELBO=-14.91
Epoch 25: ELBO=-14.84
Epoch 26: ELBO=-14.77
Epoch 27: ELBO=-14.72
Epoch 28: ELBO=-14.64
Epoch 29: ELBO=-14.64
Epoch 30: ELBO=-14.59
Epoch 31: ELBO=-14.54
Epoch 32: ELBO=-14.56
Epoch 33: ELBO=-14.55
Epoch 34: ELBO=-14.52
Epoch 35: ELBO=-14.54
Epoch 36: ELBO=-14.53
Epoch 37: ELBO=-14.52
Epoch 38: ELBO=-14.49
Epoch 39: ELBO=-14.48
Epoch 40: ELBO=-14.49
Epoch 41: ELBO=-14.48
Epoch 42: ELBO=-14.48
Epoch 43: ELBO=-14.46
Epoch 44: ELBO=-14.47
Epoch 45: ELBO=-14.44
Epoch 46: ELBO=-14.

In [ ]:
with torch.no_grad():
    print(torch.mean((gllvm0.wz - gllvm.wz)**2), torch.mean((gllvm0.wz + gllvm.wz)**2))

tensor(2.0309, device='cuda:0') tensor(0.4953, device='cuda:0')


In [ ]:
gllvm0.wz

Parameter containing:
tensor([[-0.1115,  0.1204, -0.3696, -0.2404, -1.1969]], device='cuda:0',
       requires_grad=True)

In [ ]:
gllvm.wz

Parameter containing:
tensor([[ 0.0882, -1.1704,  0.2477,  1.4034,  1.1194]], device='cuda:0',
       requires_grad=True)

In [ ]:
print("True W_z:\n", gllvm0.wz)
print("Estimated W_z:\n", gllvm.wz)

print("True bias:\n", gllvm0.bias)
print("Estimated bias:\n", gllvm.bias)

print("True scale:\n", gllvm0.scale)
print("Estimated scale:\n", gllvm.scale)


True W_z:
 Parameter containing:
tensor([[-0.1115,  0.1204, -0.3696, -0.2404, -1.1969]], device='cuda:0',
       requires_grad=True)
Estimated W_z:
 Parameter containing:
tensor([[ 0.0882, -1.1704,  0.2477,  1.4034,  1.1194]], device='cuda:0',
       requires_grad=True)
True bias:
 Parameter containing:
tensor([0., 0., 0., 0., 0.], device='cuda:0', requires_grad=True)
Estimated bias:
 Parameter containing:
tensor([ 0.0010, -0.2908, -0.0166,  0.0100,  0.0024], device='cuda:0',
       requires_grad=True)
True scale:
 tensor([1., 1., 1., 1., 1.], device='cuda:0', grad_fn=<ExpBackward0>)
Estimated scale:
 tensor([1.0000, 1.0000, 1.0000, 1.0377, 1.2141], device='cuda:0',
       grad_fn=<ExpBackward0>)
